注意点
- 現状は一問一答方式で、Chat履歴の記憶には非対応

In [2]:
import os
import re

from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.chat_models import AzureChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.utilities import BingSearchAPIWrapper

from dotenv import load_dotenv 
load_dotenv("/home/shohei/.env", override=True)


True

In [3]:
# search = SerpAPIWrapper(serpapi_api_key=os.environ["SERPAPI_API_KEY"]) # SerpAPI
search = BingSearchAPIWrapper(bing_subscription_key=os.environ["BING_SUBSCRIPTION_KEY"], # Bing API
                              bing_search_url="https://api.bing.microsoft.com/v7.0/search")

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world. The input to this should be a single search term."
    )
]

template = """Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question. Please respond in Japanese.

These were previous tasks you completed:



Begin!

Question: {input}
{agent_scratchpad}"""


# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            # raise ValueError(f"Could not parse LLM output: `{llm_output}`") #強引に正常終了させる形へ
            return AgentFinish(
                return_values={"output": llm_output},
                log=llm_output)
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)


output_parser = CustomOutputParser()

# AzureChatOpenAIクラスのインスタンス作成
llm = AzureChatOpenAI(openai_api_base = os.environ["AZURE_OPENAI_API_BASE"], # リソースによって異なる
                        openai_api_version = "2023-07-01-preview", 
                        openai_api_key = os.environ["AZURE_OPENAI_API_KEY"],
                        deployment_name="gpt-35-turbo", # モデルのデプロイ名
                        temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [4]:
agent_executor.run("2035年問題と2040年問題の違いは何ですか")



> Entering new AgentExecutor chain...
Thought: I need to find information about the differences between the 2035 problem and the 2040 problem.
Action: Search
Action Input: "differences between 2035 problem and 2040 problem"

Observation:<b>2035</b>年問題とは、2025年から<b>2035</b>年にかけて団塊世代（一般的に1947年〜1949年生まれの人々）の高齢化が進み、ほかの世代を含めた人口の約3分の1を高齢者が占めることで、医療や経済に影響を与えると想定 <b>2040</b>年問題と2054年問題 では、「<b>2040</b>年問題」とは何か。それは、2025年から<b>2040</b>年という僅か15年間において、現役人口（20歳ー64歳）が約1,000万人も減少するという問題である。既述の「将来人口推計」では、2025年に6,634万 失業率の増加 <b>2035</b>年問題の種類 人口の3割が高齢者 医療・介護費用は1.9倍に 日本の総人口は1 億人を下回る <b>2035</b>年問題の影響 <b>2035</b>年問題による経済への影響 <b>2035</b>年問題による医療への影響 <b>2035</b>年問題による介護への影響 <b>2035</b>年問題による社会への影響 <b>2035</b>年問題による教育への影響 <b>2035</b>年問題で変化する働き方 正社員が消える 時間や空間に捉われない働き方へ This <b>problem</b> is that the working population (aged <b>between</b> 20 and 64) will decrease by as much as 10 million in only 15 years <b>between</b> 2025 <b>and 2040</b>. The above-cited population projections indicate that the working pop

'I have found information about the differences between the 2035 problem and the 2040 problem. The 2035 problem refers to the aging population of the baby boomer generation, where approximately one-third of the population will be elderly by 2035. This will have implications for healthcare and the economy. On the other hand, the 2040 problem is the significant decrease in the working population (aged 20-64) within a 15-year period from 2025 to 2040, with a projected decrease of 10 million people. This will also have various impacts on society, including changes in employment patterns and the need for caregiving.'

In [6]:
agent_executor.run("先ほどの回答を日本語訳してください")



> Entering new AgentExecutor chain...
Thought: I need to translate the previous answer into Japanese.
Action: Search
Action Input: "Translate English to Japanese"

Observation:Translation Google&#39;s service, offered free of charge, instantly translates words, phrases, and web pages between <b>English</b> and over 100 other languages. Millions <b>translate</b> with DeepL every day. Popular: Spanish to <b>English</b>, French to <b>English</b>, and <b>Japanese</b> to <b>English</b>. Millions <b>translate</b> with DeepL every day. Popular: Spanish to <b>English</b>, French to <b>English</b>, and <b>Japanese</b> to <b>English</b>. <b>Translate</b> from <b>English</b> to <b>Japanese</b> online - a free and easy-to-use translation tool. Simply enter your text, and Yandex <b>Translate</b> will provide you with a quick and accurate translation in seconds. Try Yandex <b>Translate</b> for your 英語翻訳のやり方 1.「原文」のテキストボックスに、英訳や和訳したい文章を入力してください。 2.翻訳ボタンをクリックしてください。 3.英語の場合は和訳された文章が、日本語の場合は英訳された翻訳結果

'先ほどの回答を日本語訳してください (Please translate the previous answer into Japanese)'

Refs
https://python.langchain.com/docs/modules/agents/how_to/custom_llm_chat_agent